In [75]:
import json
from math import sqrt

# Reading Train Data

In [76]:
with open('./data/train.json') as d:
    json_data = json.load(d)

In [78]:
def get_doc_terms(doc_info):
    body_terms = data['body'].split()
    title_terms = data['title'].split()
    all_terms = body_terms + title_terms
    return all_terms

In [79]:
vectors = {}
query = {}

for index, data in enumerate(json_data):

    index = str(index)
    vectors[index] = {
        'terms': {},
        'category': 0
    }
    
    all_terms = get_doc_terms(data)
    
    for term in all_terms:
        if term in vectors[index]['terms']:
            vectors[index]['terms'][term] += 1
        else:
            vectors[index]['terms'][term] = 1
    
    vectors[index]['category'] = data['category']

# KNN

We should calculate euclidean distance nd cosine similarity for each document with respect to query. then return top k of them.

## Cosine Similarity

 consine = a.b / |a|.|b|

In [8]:
def cosine_similarity(q, v):
    cosine_similarity_value = 0 
    numerator = 0
    
    for term in q:
        if term in v:
            numerator += q[term] * v[term]

    q_size = 0    
    for term in q:
        q_size += q[term] * q[term]
    q_size = sqrt(q_size)
    
    v_size = 0
    for term in v:
        v_size += v[term] * v[term]
    v_size = sqrt(v_size)
    
    return numerator / (v_size * q_size)

def euclidean_distance(q, v):
    distance = 0
    
    for term in q:
        if term in v:
            distance += (v[term] - q[term]) ** 2
        else:
            distance += q[term] ** 2
    
    for term in v:
        if term not in q:
            distance += v[term] ** 2
    
    return sqrt(distance)

    
def KNN(query_info, k = 1):
    query_terms = get_doc_terms(query_info)
    
    for term in query_terms:
        if term in query:
            query[term] += 1
        else:
            query[term] = 1
    
    scores = {
        'consine': {},
        'euclidean': {}
    }
    for doc_id in vectors:
        doc_id = str(doc_id)
        scores['consine'][doc_id] = consine_similarity(query_terms, vectors[doc_id][terms])
        scores['euclidean'][doc_id] = euclidean_distance(query_terms, vectors[doc_id][terms])
        
#     return scores

# Naive Bayes



In [69]:
words_occur_class = {} #Occurance of each word in each class

def get_word_occur_class(category_no = 4):
    for doc_id in vectors:
        category_id = str(vectors[doc_id]['category'])
        terms = vectors[doc_id]['terms']
        
        for term in terms:    
            if term not in words_occur_class:
                words_occur_class[term] = {}
                
                for i in range(1, category_no + 1):
                    words_occur_class[term][str(i)] = 0
            
            words_occur_class[term][category_id] += terms[term]            
                
get_word_occur_class()


def get_category_count(category_no = 4):
    category_count = {}
    
    for i in range(1, category_no + 1):
        category_count[str(i)] = 0
    
    for doc_id in vectors:
        category_id = str(vectors[doc_id]['category'])
        category_count[category_id] += 1
    
    return category_count

def naive_bayes_training(category_no=4):
    N = len(json_data)
    category_count = get_category_count()
    

{'1': 6000, '2': 6000, '3': 6000, '4': 6000}